# 为cython调用提供接口

cython虽然必须依赖python或C/C++才能真正实现功能,但作为一个编程语言,它也需要组织可以有结构.在语法部分我们也简单介绍了下`cimport`导入语法.它是cython模块化的关键.

为cython调用提供接口可以分为:

+ 声明接口
    + 声明内部接口
    + 声明外部包装接口
+ 模块调用
    + 外部模块调用
    + 内部子模块调用

In [1]:
%load_ext cython

## 声明内部接口

内部接口的声明包括:

+ 声明类型别名
+ 声明C全局变量
+ 声明C函数
+ 声明扩展类型

尤其是结合纯净模式,我们可以做到渐进式的优化模块性能.

无论哪种声明,它必须与实现部分一一对应--比如有个实现`A.pyx`,那它的声明文件即为`A.pxd`,`A.pyx`中有一个函数为`cdef void func(int a)`那如果我们希望将它暴露给其他cython模块就需要在`A.pyx`中也有一个`cdef void func(int a)`,当然如果我们并不想将它暴露给其他cython模块就不要在`A.pyx`中有对应的声明.这个逻辑基本和C语言的头文件是一致的.

### 声明类型别名

Cython中我们可以使用关键词`ctypedef`来声明类型别名,其语法为`ctypedef 类型 类型别名`,比如将`unsigned int`定义为`uint_t`可以写作

+ `A.pxd`

    ```cython
    ctypedef unsigned int uint_t
    ```

`ctypedef`可以也仅可以用于任何**C类型**,例如整数,浮点数,struct,指针等.通常用于提高可读性和对模块使用场景的匹配程度.

通常别名声明既可以放在`.pxd`文件也可以放在实现部分,这取决于如下几点:

1. 你是否希望外部cython模块使用这个别名,如果希望则放在`.pxd`文件,不希望则放实现部分
2. 你的对外接口是否有用到这个别名,如果有用到则放在`.pxd`文件,没有则放实现部分

一旦你在`.pxd`文件中声明了别名,那实现部分就不需要重复声明了.

### 声明C变量

你可以直接用`cdef 类型 变量名`声明一个变量,比如

+ `A.pxd`

    ```cython
    cdef int GLOBAL_VAL
    ```

通常变量声明既可以放在`.pxd`文件也可以放在实现部分,这取决于你是否希望外部cython模块使用这个变量,如果希望则放在`.pxd`文件,不希望则放实现部分.

一旦你在.pxd文件中声明变量,那实现部分就不需要重复声明了可以直接赋值.比如上面声明文件对应的实现文件中可以直接这样赋值

+ `A.pyx`

    ```cython
    GLOBAL_VAL = 100
    ```

### 声明C函数

我们可以使用`cdef [返回值类型] 函数名(形参类型 形参名,...) [noexcept | except 异常标志量]`来声明一个C函数;`cpdef 返回值类型 函数名(形参类型 形参,...)`来声明一个Python和C都可调用的函数

需要注意的是如果参数有默认值,这个默认值在实现部分设置,但`.pxd`文件的声明中要使用`=*`来表示这个参数有默认值

+ `A.pxd`

    ```cython
    cdef uint_t mycfunc(uint_t x, uint_t y=*)

    cpdef int mycpfunc(int x, int y=*)
    ```
    
+ `A.pyx`

    ```cython
    cdef uint_t mycfunc(uint_t x, uint_t y=2):
        cdef uint_t a
        a = x-y
        return a + x * y

    cpdef int mycpfunc(int x, int y=2):
        cdef int a
        a = x+y
        return a + x * y

    ```
    
**特别需要注意的是**:python函数不能在`.pxd`中声明,**包括静态化参数的python函数**.

通常函数声明既可以放在`.pxd`文件也可以放在实现部分,这取决于你是否希望外部cython模块使用这个函数,如果希望则放在.pxd文件,不希望则放实现部分.

#### 声明内联函数

在`.pyd`文件中同样可以写内联函数.但要注意内联函数除了要声明还要在`.pyd`中有实现

```cython
cdef inline int int_min(int a, int b):
    return b if b < a else a
```

## 声明扩展类型

扩展类型中只需要且只能声明其中的C方法,Python和C都可调用的方法以及C属性,python部分不能声明.

+ `A.pxd`

```cython
cdef class A:
    cdef public int a
    cdef int b
    cdef double foo(self,double x)
    cpdef double bar(self, double x)
```

其中C属性部分在`.pxd`中声明了就不需要再在实现部分声明了,只要正常赋值即可.

+ `A.pyx`

```cython
cdef class A:
    def __init__(self, b:int = 0):
        self.a = 3
        self.b = b

    cdef double foo(self, double x):
        return x + _helper(1.0)
    
    cpdef double bar(self, double x):
        return x**2 + _helper(1.0)

    def foobar(self,x:float)->float:
        return x**2 + _helper(1.0)
```


这块的演示代码在[]()中

### 与纯净模式结合使用

普通cython语法具有更好的表达能力,代码可读性更好功能也最全.但纯净模式与声明文件`.pyd`结合使用更加适合渐进式的性能优化工作流.我们可以先使用纯python实现,然后根据需要在对原始python代码做出极小改动的情况下使用声明文件`.pyd`将其改造为cython模块.

cython在编译时会将声明文件`.pyd`中声明的变量,函数,方法,等与源文件中进行匹配,在纯净模式下,由于有声明文件`.pyd`中的声明,即便实现文件中定义的是python函数python类,也可以被当做声明成的C函数扩展类来进行编译.


#### 纯净模式改写原例子 

我们只需要改写实现部分:

+ `A.py`

```cython
# cython: embedsignature=True
# cython: embedsignature.format=python
import cython

if cython.compiled:
    print("Yep, I'm compiled.")

else:
    print("Just a lowly interpreted script.")
    uint_t = int
    def int_min(a: int, b: int) -> int:
        return b if b < a else a

GLOBAL_VAL = 100

# uint_t = cython.typedef(cython.uint)


def mycfunc(x: uint_t, y: uint_t = 2) -> uint_t:
    a: uint_t = x - y
    return a + x * y


def mycpfunc(x: cython.int, y: cython.int = 2) -> int:
    a: cython.int = x + y
    return a + x * y


@cython.locals(x=cython.int, y=cython.int)
def mypfuncstatic(x: int, y: int) -> int:
    return x * y


def mypfunc(x: int, y: int) -> int:
    return 2 * x * y


def _helper(a: cython.double) -> float:
    return a + 1


class A:
    def __init__(self, b: int = 0):
        self.a = 3
        self.b = b

    def foo(self, x: cython.double) -> cython.double:
        return x + _helper(1.0)

    def bar(self, x: float) -> float:
        return x**2 + _helper(1.0)

    def foobar(self, x: float) -> float:
        return x**2 + _helper(1.0)
```

通常用纯净模式都是要考虑不编译情况下纯python代码的可执行性的,因此需要注意的有如下几点:

1. 声明文件中的类型别名在cython中可以被直接使用,但在python下不行,需要像例子中一样在检查到没有编译的情况下提前声明别名
2. 推荐无论是C函数,python函数还是C和python都可以调用的函数都保留typehints方便维护
3. 建议C函数,C和python都可以调用的函数在typehints中使用`cython`模块中的类型进行声明;不静态化参数的python函数在typehints中使用python;静态化参数的python函数必须在typehints中使用`cython`模块中的类型进行声明
4. 在给python写扩展的情况下建议非python接口的函数以及方法遵循python社区西关使用`_`开头命名


这块的演示代码在[]()中

## 作为cython模块的模块化

作为cython模块,它的主要作用是让cython代码间可以模块化编程,从而实现抽象和复用,是配合`cimport`语法的的代码组织工具.比如我们可以写一个cython模块A,当在一个新的cython项目中需要用到A中定义过的C函数或扩展类型中定义的C方法时我们就可以用`cimport`语法导入这个模块并使用了.因此cython模块可以看做是作为python模块的模块化和作为C模块的模块化的前提,后两者是前者的应用.只有弄明白如何作为cython模块进行模块化才能在后两种环境下进行的下去.而cython作为一个python和C之间的桥接语言,cython模块是无法脱离python或C使用的,也就不存在编译一说.

cython模块主要是看声明文件`.pxd`,和python的模块系统非常类似,一个cython项目如果要作为cython模块必须满足两种形式中的一种:

+ 简单模块,可以是一个单独的`.pxd`
+ 复杂模块,可以是一个带`__init__.pxd`的文件夹.

cython的复杂模块允许内部继续包含子模块,以我们的例子`mymath`为例:

```txt
mymath---\
    |---__init__.pxd
    |---normalize_and_l2norm.pyx # 示例内部调用
    |---normalize_and_l2norm.pxd # 示例内部调用的声明
    |---inner---\
    |       |---__init__.pxd
    |       |---l2norm.pxd   # 有接口`l2norm`
    |       |---l2norm.pyx
    |
    |---median_along_axis0.pxd  # 有接口`_median_along_axis0`
    |---median_along_axis0.pyx
    |---normalize.pxd  # 有接口`_normalize`
    |---normalize.pyx
    |---notexist.pxd #示例有声明无实现
    ...
cythoncallmymath.pyx # 示例外部调用
cythoncallmymath.py # 示例外部调用纯净模式
```

### 内部调用

在其中的子模块可以和python中一样,使用相对引用导入需要的接口,如果是纯净模式则还是只能老老实实绝对引用导入,比如在一个新实现文件`mymath.pyx`中要导入接口`l2norm`和`_normalize`:

+ `normalize_and_l2norm.pyx`(cython语法)

```cython
...
from .inner.l2norm cimport l2norm
from .normalize cimport _normalize
...
```

+ `normalize_and_l2norm.py`(纯净模式语法)

```python
from cython.cimports.mymath.inner.l2norm import l2norm
from cython.cimports.mymath.normalize import _normalize
```

当然了Cython中同样要避免钻石引用.

### 外部调用

对于外部调用,无论哪种语法都只能老老实实绝对调用

+ `cythoncallmymath.pyx`(cython语法)

    ```cython
    from mymath.normalize cimport _normalize
    import numpy as np


    def callmymath():
        cdef double[:] output = _normalize(np.array([1.1,2.2,3.3,4.4]))
        print(np.asarray(output))
    ```

+ `cythoncallmymath.py`(纯净模式语法)

```cython
import cython
from cython.cimports.mymath.normalize cimport _normalize
import numpy as np


def callmymath():
    output: cython.double[:] = _normalize(np.array([1.1,2.2,3.3,4.4]))
    print(np.asarray(output))
```